In [2]:
from datetime import datetime, date

import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv("./datasets/BPD_Part_1_Victim_Based_Crime_Data.csv")



# TODO: Update to MASTER Redacted.
# df = pd.read_csv("./datasets/Redacted YTD_MASTER_CRIME 2012_2017.csv")


In [5]:
df.head(10)

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,Total Incidents
0,03/10/2018,23:38:00,4E,3100 STAFFORD ST,COMMON ASSAULT,I,HANDS,832.0,SOUTHWESTERN,Gwynns Falls,-76.66895,39.27941,"(39.27941, -76.66895)",ROW/TOWNHO,1
1,03/10/2018,23:38:00,4E,3100 STAFFORD ST,COMMON ASSAULT,I,HANDS,832.0,SOUTHWESTERN,Gwynns Falls,-76.66895,39.27941,"(39.27941, -76.66895)",ROW/TOWNHO,1
2,03/10/2018,23:00:00,6D,700 E 41ST ST,LARCENY FROM AUTO,O,NaN,542.0,NORTHERN,Pen Lucy,-76.60616,39.33973,"(39.33973, -76.60616)",STREET,1
3,03/10/2018,22:53:00,4E,2100 W NORTH AVE,COMMON ASSAULT,NaN,HANDS,741.0,WESTERN,Easterwood,-76.65153,39.30957,"(39.30957, -76.65153)",NaN,1
4,03/10/2018,22:15:00,4E,3400 GREENMOUNT AVE,COMMON ASSAULT,I,HANDS,543.0,NORTHERN,Waverly,-76.60941,39.33009,"(39.33009, -76.60941)",OTHER - IN,1
5,03/10/2018,21:50:00,4C,800 N WOLFE ST,AGG. ASSAULT,I,OTHER,321.0,EASTERN,Middle East,-76.59101,39.29972,"(39.29972, -76.59101)",OTHER - IN,1
6,03/10/2018,21:50:00,4C,22ND ST & N CHARLES ST,AGG. ASSAULT,O,OTHER,514.0,NORTHERN,Old Goucher,-76.61675,39.31424,"(39.31424, -76.61675)",STREET,1
7,03/10/2018,21:40:00,3AK,POPPLETON ST & WASHINGTON BL,ROBBERY - STREET,O,KNIFE,932.0,SOUTHERN,Washington Village/Pigtow,-76.63179,39.28287,"(39.28287, -76.63179)",STREET,1
8,03/10/2018,21:15:00,6J,200 BALLOU CT,LARCENY,I,NaN,211.0,SOUTHEASTERN,Perkins Homes,-76.59728,39.28848,"(39.28848, -76.59728)",ROW/TOWNHO,1
9,03/10/2018,20:47:00,4F,3000 WESTWOOD AVE,ASSAULT BY THREAT,I,NaN,813.0,SOUTHWESTERN,Northwest Community Actio,-76.66818,39.30803,"(39.30803, -76.66818)",ROW/TOWNHO,1


In [8]:
start = datetime(2016, 1, 1, 0, 0, 0)
end = datetime(2016, 12, 31, 23, 59, 59)

print(start)
print(end)


2016-01-01 00:00:00
2016-12-31 23:59:59


In [11]:
# Preprocess crime data from BDP Dataset

crime_datetime = []
crime_descript = []
crime_district = []
crime_neighbor = []


# Iterate through all rows, attempt to get datetime columns parsed and working
for index, row in df.iterrows():
    datetime_str = "{} {}".format(row["CrimeDate"], row["CrimeTime"])

    worked_first_parse = False
    
    try:
        datetime_processed = datetime.strptime(datetime_str, '%m/%d/%Y %H:%M:%S')
        worked_first_parse = True
        
    except ValueError as e:
        pass

    if not worked_first_parse:
        try:
            datetime_processed = datetime.strptime(datetime_str, '%m/%d/%Y %H%M')
        except ValueError as e:
            continue

    if start <= datetime_processed <= end:
        crime_datetime.append(datetime_processed)
        crime_descript.append(row["Description"])
        crime_district.append(row["District"])
        crime_neighbor.append(row["Neighborhood"])

    if datetime_processed < start:
        break

# Append to a new dataframe
crime = {
    'Datetime': crime_datetime,
    'Description': crime_descript,
    'District': crime_district,
    'Neighborhood': crime_neighbor    
}

crime_df = pd.DataFrame(data=crime)

In [13]:
# print(len(df["CrimeDate"]))
crime_df.head(25)

,Datetime,Description,District,Neighborhood
0,2016-05-09 21:59:00,ROBBERY - STREET,SOUTHERN,NaN
1,2016-05-09 21:59:00,ROBBERY - STREET,SOUTHERN,NaN
2,2016-05-09 21:59:00,ROBBERY - STREET,SOUTHERN,NaN
3,2016-05-09 21:59:00,ROBBERY - STREET,SOUTHERN,NaN
4,2016-05-09 21:59:00,ROBBERY - STREET,SOUTHERN,NaN
5,2016-05-04 11:00:00,LARCENY FROM AUTO,EASTERN,NaN
6,2016-12-31 23:51:00,AGG. ASSAULT,EASTERN,Darley Park
7,2016-12-31 23:30:00,COMMON ASSAULT,NORTHWESTERN,Central Park Heights
8,2016-12-31 23:30:00,LARCENY FROM AUTO,SOUTHEASTERN,Canton
9,2016-12-31 23:30:00,LARCENY,EASTERN,CARE
